# Run script - gridded data to catchment timeseries

In this script we extract catchment timeseries of precipitation, potential evaporation and temperature from global gridded products. 

This scripts only works in the conda environment **sr_env**. In this environment all required packages are available. If you have **not** installed and activated this environment before opening this script, you should check the installation section in the *README* file. 

### 1. Getting started
First, import all the required packages.

In [1]:
# import packages
import glob
from pathlib import Path
import os
import geopandas as gpd
import iris
import iris.pandas
import numpy as np
from esmvalcore import preprocessor
from iris.coords import DimCoord
from iris.cube import Cube
from pathos.threading import ThreadPool as Pool
from datetime import datetime
from datetime import timedelta
import pandas as pd

Here we import all the python functions defined in the scripts *f_grid_to_catchments.py*.

In [2]:
# import python functions
from f_grid_to_catchments import *

### 2. Define working and data directories
Here we define the working directory, where all the scripts and output are saved.

We also definet he data directory where you have the following subdirectories:

/data/forcing/*netcdf forcing files*\
/data/shapes/*catchment shapefiles*\
/data/gsim_discharge/*gsim discharge timeseries*


In [3]:
# Check current working directory (helpful when filling in work_dir below)
os.getcwd()

'/work/users/vanoorschot/fransje/scripts/GLOBAL_SR/global_sr_module/scripts'

In [4]:
# define your script working directory
# work_dir=Path("/home/fransjevanoors/global_sr_module")
work_dir=Path("/work/users/vanoorschot/fransje/scripts/GLOBAL_SR/global_sr_module")
# define your data directory
data_dir=Path("/work/users/vanoorschot/fransje/scripts/GLOBAL_SR/global_sr_module/data")


### 3. From gridded data to catchment timeseries
We don't have data on precipitation, potential evaporation and temperature at the catchment scale. Therefore, we use global gridded products of these parameters (there are a lot of possibilities which data to use). For doing analyses at the catchment scale, we need to convert these gridded products into catchment timeseries.
To do this, we calculate the mean parameter values of the gridcells that fall within the catchment shapes. The procedure is defined by the functions in the *f_grid_to_catchments.py* file.

In [5]:
# make output directories
if not os.path.exists(f'{work_dir}/output/forcing_timeseries/raw'):
    os.makedirs(f'{work_dir}/output/forcing_timeseries/raw')
if not os.path.exists(f'{work_dir}/output/forcing_timeseries/processed'):
    os.makedirs(f'{work_dir}/output/forcing_timeseries/processed')

In [6]:
# define directories 
SHAPE_DIR = Path(f'{data_dir}/shapes/') # dir of shapefiles
NC4_DIR = Path(f'{data_dir}/forcing/') # dir of netcdf forcing files
OUT_DIR = Path(f'{work_dir}/output/forcing_timeseries/raw') # output dir

The conversion from grid to catchment is computationally expensive. Therefore, we run this conversion for all catchments in parallel using the python function *pathos threadpool* (https://pathos.readthedocs.io/en/latest/pathos.html#module-pathos.threading).
With the *construct_lists_for_parallel_function* function from *f_grid_to_catchments.py* we create lists that contain all combinations of shapefile, netcdf-file and output-directory. OPERATOR LIST NOT NEEDED?. These lists are the input for the *run_function_parallel* function from *f_grid_to_catchments.py*, which returns timeseries of the catchment mean values of precipitation (P), potential evaporation (Ep) and  temperature (T). 

In [7]:
# Construct lists for parallel run
(shapefile_list, netcdf_list, operator_list, output_dir_list) = construct_lists_for_parallel_function(NC4_DIR, SHAPE_DIR, OUT_DIR)

In [8]:
# run function parallel
run_function_parallel(shapefile_list, netcdf_list, operator_list, output_dir_list)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

The output of the *run_function_parallel* function contains daily timeseries of P, Ep and T for all catchments. Here we postprocess these data to get dataframes containing Ep, P and T together with daily, monthly and yearly timeseries, climatology and mean values (stored as *csv* files). This postprocessing is done in the *process_forcing_timeseries* function defined in *f_grid_to_catchments.py*.

In [9]:
# define input directory
fol_in=f'{work_dir}/output/forcing_timeseries/raw'
# define output directory
fol_out=f'{work_dir}/output/forcing_timeseries/processed'

# get catch_id_list
catch_id_list = np.genfromtxt(f'{work_dir}/output/catch_id_list.txt',dtype='str')

# define variables
var = ['Ep','P','T']

# run process_forcing_timeseries (defined in f_grid_to_catchments.py) for all catchments in catch_id_list
for catch_id in catch_id_list:
    process_forcing_timeseries(catch_id,fol_in,fol_out,var)

In [10]:
# print P Ep T timeseries for catchment [0] in catch_id_list
catch_id = catch_id_list[0]
f = glob.glob(f'{fol_out}/daily/{catch_id}*.csv')
print(f)
c = pd.read_csv(f[0], index_col=0)
c.head()

['/work/users/vanoorschot/fransje/scripts/GLOBAL_SR/global_sr_module/output/forcing_timeseries/processed/daily/br_0000495_1995_2000.csv']


,Ep,P,T
time,,,
1995-01-01,3.011033,0.004925,28.862823
1995-01-02,3.397685,0.000000,28.742493
1995-01-03,3.092484,0.000000,29.230500
1995-01-04,3.917221,0.000000,28.689178
1995-01-05,4.295576,0.019365,28.674103
